## Problem 2
Given the dataset in DailyPrices.csv, you have a portfolio of
- 100 shares of SPY
- 200 shares of AAPL
- 150 shares of EQIX
1. Calculate the current value of the portfolio given today is 1/3/2025
2. Calculate the VaR and ES of each stock and the entire portfolio at the 5% alpha level
assuming arithmetic returns and 0 mean return, for the following methods:
- a. Normally distributed with exponentially weighted covariance with lambda=0.97
- b. T distribution using a Gaussian Copula
- c. Historic simulation using the full history.
3. Discuss the differences between the methods

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.optimize import minimize

# Load dataset
file_path = "DailyPrices.csv"
df = pd.read_csv(file_path, parse_dates=["Date"])
df.set_index("Date", inplace=True)

# Select relevant stocks
assets = ["SPY", "AAPL", "EQIX"]
df = df[assets].dropna()

# Compute log returns
returns = np.log(df / df.shift(1)).dropna()

# Define portfolio holdings
holdings = {"SPY": 100, "AAPL": 200, "EQIX": 150}
portfolio_value = sum(holdings[asset] * df.loc["2025-01-03", asset] for asset in assets)
print(f"Portfolio Value on 2025-01-03: ${portfolio_value:,.2f}")

# Compute exponentially weighted covariance matrix (lambda = 0.97)
lambda_ = 0.97
cov_matrix = returns.ewm(span=(2 / (1 - lambda_)) - 1).cov().iloc[-3:]

# Compute VaR and ES (Normal Distribution) for each stock and portfolio
alpha = 0.05
z_alpha = stats.norm.ppf(alpha)
var_normal_assets = {}
es_normal_assets = {}

for asset in assets:
    vol = returns[asset].std()
    var_normal_assets[asset] = -z_alpha * vol
    es_normal_assets[asset] = -stats.norm.pdf(z_alpha) / alpha * vol

portfolio_weights = np.array([holdings[asset] * df.loc["2025-01-03", asset] / portfolio_value for asset in assets])
portfolio_vol = np.sqrt(portfolio_weights @ cov_matrix.values @ portfolio_weights.T)
var_normal = -z_alpha * portfolio_vol
es_normal = -stats.norm.pdf(z_alpha) / alpha * portfolio_vol

# Fit t-distribution to each stock's returns
params = {asset: stats.t.fit(returns[asset]) for asset in assets}

# Convert returns to uniform scale using the fitted t-distributions
uniform_returns = returns.copy()
for asset in assets:
    df_, loc_, scale_ = params[asset]
    uniform_returns[asset] = stats.t.cdf(returns[asset], df=df_, loc=loc_, scale=scale_)

# Compute correlation matrix for copula
corr_matrix = uniform_returns.corr()

# Generate multivariate normal samples
num_samples = 10000
normal_samples = np.random.multivariate_normal(mean=[0, 0, 0], cov=corr_matrix, size=num_samples)

# Convert normal samples back to uniform using CDF
uniform_simulated = stats.norm.cdf(normal_samples)

# Convert uniform back to t-distribution returns
simulated_returns = uniform_simulated.copy()
for i, asset in enumerate(assets):
    df_, loc_, scale_ = params[asset]
    simulated_returns[:, i] = stats.t.ppf(uniform_simulated[:, i], df=df_, loc=loc_, scale=scale_)

# Compute VaR and ES for each stock using Copula method
var_copula_assets = {}
es_copula_assets = {}
for i, asset in enumerate(assets):
    var_copula_assets[asset] = -np.percentile(simulated_returns[:, i], alpha * 100)
    es_copula_assets[asset] = -simulated_returns[:, i][simulated_returns[:, i] <= -var_copula_assets[asset]].mean()

# Compute portfolio returns using simulated data
simulated_portfolio = simulated_returns @ portfolio_weights
var_copula = -np.percentile(simulated_portfolio, alpha * 100)
es_copula = -simulated_portfolio[simulated_portfolio <= -var_copula].mean()

# Compute VaR and ES using historical simulation for each stock
var_historical_assets = {}
es_historical_assets = {}
for asset in assets:
    var_historical_assets[asset] = -np.percentile(returns[asset], alpha * 100)
    es_historical_assets[asset] = -returns[asset][returns[asset] <= -var_historical_assets[asset]].mean()

# Compute historical simulation for the portfolio
historical_portfolio = returns @ portfolio_weights
var_historical = -np.percentile(historical_portfolio, alpha * 100)
es_historical = -historical_portfolio[historical_portfolio <= -var_historical].mean()

# Output results
print("\nNormal Distribution Method:")
for asset in assets:
    print(f"{asset} VaR: ${var_normal_assets[asset] * holdings[asset] * df.loc['2025-01-03', asset]:,.2f}")
    print(f"{asset} ES: ${es_normal_assets[asset] * holdings[asset] * df.loc['2025-01-03', asset]:,.2f}")
print(f"Portfolio VaR: ${var_normal * portfolio_value:,.2f}")
print(f"Portfolio ES: ${es_normal * portfolio_value:,.2f}")

print("\nCopula Method with t-distribution:")
for asset in assets:
    print(f"{asset} VaR: ${var_copula_assets[asset] * holdings[asset] * df.loc['2025-01-03', asset]:,.2f}")
    print(f"{asset} ES: ${es_copula_assets[asset] * holdings[asset] * df.loc['2025-01-03', asset]:,.2f}")
print(f"Portfolio VaR: ${var_copula * portfolio_value:,.2f}")
print(f"Portfolio ES: ${es_copula * portfolio_value:,.2f}")

print("\nHistorical Simulation Method:")
for asset in assets:
    print(f"{asset} VaR: ${var_historical_assets[asset] * holdings[asset] * df.loc['2025-01-03', asset]:,.2f}")
    print(f"{asset} ES: ${es_historical_assets[asset] * holdings[asset] * df.loc['2025-01-03', asset]:,.2f}")
print(f"Portfolio VaR: ${var_historical * portfolio_value:,.2f}")
print(f"Portfolio ES: ${es_historical * portfolio_value:,.2f}")

Portfolio Value on 2025-01-03: $251,862.50

Normal Distribution Method:
SPY VaR: $786.55
SPY ES: $-986.37
AAPL VaR: $1,076.50
AAPL ES: $-1,349.97
EQIX VaR: $3,616.78
EQIX ES: $-4,535.59
Portfolio VaR: $3,892.41
Portfolio ES: $-4,881.24

Copula Method with t-distribution:
SPY VaR: $721.26
SPY ES: $1,008.64
AAPL VaR: $962.98
AAPL ES: $1,388.15
EQIX VaR: $3,389.79
EQIX ES: $4,957.79
Portfolio VaR: $4,197.49
Portfolio ES: $6,114.05

Historical Simulation Method:
SPY VaR: $820.93
SPY ES: $1,032.46
AAPL VaR: $1,007.60
AAPL ES: $1,388.98
EQIX VaR: $3,545.28
EQIX ES: $4,660.30
Portfolio VaR: $4,364.04
Portfolio ES: $5,887.27
